Write a python code using Pandas to load the Transaction2.csv and the subject.xlsx datasets. Then, write the code needed to perform the following modifications:
1. Introduce in the subjects Dataframe the dichotomic variable ‘largecommitter’. It is set to True, if the subject has committed more than three transactions. False, otherwise
2. Introduce in the Transaction2 Dataframe a numeric variable 'charge', to be calculated in the following the way:
- 3% of the amount, if the transaction is of type ‘credit card’
- 3 euro, if the transaction is of type ‘bank transfer’
- 0, otherwise \\
Print on screen the overall count of received transactions and the overall amount of taxes paid by each recipient, by summing the amount of all taxes paid on the transactions  received \\
Save on an excel file a table reporting, for each subject, her id number, her first and last name and the overall amount of. taxes she paid

#Prep:

In [ ]:
#first off we need to import panda
import pandas as pd

In [ ]:
#in order to make t
t = pd.read_csv('Transactions2.csv', sep = ';', dtype = {'recipient' : str, 'committer': str})

t.rename(columns = {'Unnamed: 0': 'Id', 'amunt': 'amount'}, inplace= True)
t['charge'] = 0

In [ ]:
t.dtypes

Id            int64
amount        int64
recipient    object
committer    object
type         object
charge        int64
dtype: object

In [ ]:
#working on s
s = pd.read_excel('subjects.xlsx', dtype = {'ID': str})

#also

s.loc[s['GENDER'] == 'Male', 'GENDER'] = 'M'
s.loc[s['GENDER'] == 'Female', 'GENDER'] = 'F'

#also, I would rather work with one birth column as well for both birthdays and address
s['BIRTHDATE'] = 'na'

s['BIRTHDATE'] = pd.to_datetime(s['BIRTHDAY'].astype(str) + '/'  + s['BIRTHMONTH'].astype(str) + '/' + s['BIRTHYEAR'].astype(str))
s.drop(columns = ['BIRTHYEAR', 'BIRTHMONTH', 'BIRTHDAY'], inplace = True)
s.drop(columns = ['LATITUDE', 'LONGITUDE'], inplace = True)
s.drop(columns = ['OLDID'], inplace = True)

In [ ]:
## First Part of second Excercise
t.loc[t['type']== 'bank transfer', 'charge'] = 3
t.loc[t['type']== 'credit card', 'charge'] = t['amount']*(3/100)
t.loc[t['type']== 'cash', 'charge'] = 0
t.loc[t['type']== 'cheque', 'charge'] = 0

In [ ]:
t = t.fillna('nd')
t

In [ ]:
t1 = pd.merge(t, s, left_on = 'committer', right_on = 'ID')
t1

# Actual HW

In [ ]:
#working on task one
t2 =t1.groupby(['committer'])['amount'].count()
t2

In [ ]:
###keep in mind that there's no correspondance between the index of the ids in
#the previous cell

t2 =t1.groupby(['committer'])['amount'].count()
LargeCommitter = []

for i in t['committer']:
  if i in t2[t2 >= 3]:
    LargeCommitter.append(True)
  else:
    LargeCommitter.append(False)

print(LargeCommitter)

In [ ]:
lc = pd.Series(LargeCommitter)
lc

In [ ]:
pd.merge(lc.rename('LARGECOMMITTERS'), s, left_index=True, right_index=True)


In [ ]:
#printing the sum of all amounts

t3 = t1.groupby(['recipient'])['amount'].sum()
t3


In [ ]:
#printing the charges by recipient
#I'm now using t instead of t1 given that, with the lost of the merged values,
#There were no credit cards left to work on

rxc = t.groupby(['recipient'])['charge'].sum()
rxc = pd.DataFrame(rxc)
rxct = pd.merge(rxc, s, left_on = 'recipient', right_on ='ID')
rxct

In [ ]:
#in order to print appropriate csv

rxct.drop(columns = ['GENDER', 'STREET ADDRESS', 'CAP', 'CITY', 'BIRTHDATE'], inplace = True)
rxct

In [ ]:
rxct.to_csv('SumOfTaxesF', sep = ';')

#the csv is not full of all the subjects giving the final merge, basically
#as our program can't id those people, we are left with the data about just the
#ones we can name.